# Script for PSF downsampling
### *19/05/25*

Neater than the 11/04/25 one in `PSF_shenanigans`. If you want to look at that one it's in `sanity_booklet/HE_calibration/PSF_shenanigans`

In [17]:
import os
import sys

from pathlib import Path
from glob      import glob
from functools import reduce


sys.path.append("/gluster/data/next/software/IC_311024/")
sys.path.append(os.path.expanduser('~/code/eol_hsrl_python'))
#sys.path.append("/home/e78368jw/Documents/NEXT_CODE/next_misc/")
os.environ['ICTDIR']='/gluster/data/next/software/IC_311024/'

import matplotlib.pyplot as plt
from invisible_cities.io.dst_io import load_dst
from invisible_cities.io.dst_io import df_writer
from invisible_cities.core                import tbl_functions   as tbl


import numpy  as np
import pandas as pd
import tables as tb



def load_psf(file):
    return load_dst(file, "PSF", "PSFs")

In [4]:
def downsample_PSF(PSF_c, binning):
    # apply rebinning

    PSF = PSF_c.copy(deep = True)

    PSF['xr'] = (PSF['xr'] // binning) * binning
    PSF['yr'] = (PSF['yr'] // binning) * binning

    psf_downsampled = PSF.groupby(['xr', 'yr', 'zr', 'x', 'y', 'z']).agg({
                                                          'factor' : 'mean',
                                                          'nevt'   : 'sum'}).reset_index()
    return psf_downsampled

In [5]:
def write_psf(PSF, path, name):
    
    # save PSF in correct format
    compression = 'ZLIB4'
    with tb.open_file(f'{path}/{name}', "w", filters = tbl.filters(compression)) as h5out:

        df_writer(h5out,   PSF, "PSF", "PSFs" , compression="ZLIB4")


# DEFINE YOUR PSF LOCATION FOR DOWNSAMPLING HERE

Uses current formatting as of 19/05/25

In [19]:

###################################################
# Where you're taking it from
RUN_NUMBER = 14914
TIMESTAMP  = '010525'
downsample = 2.5
# this one idk, maybe we need to standardise it
PSF_name   = f'{TIMESTAMP}_full_25us.psf'

###################################################
# Where you're putting it
OUTPUT_TS  = '190525'
OUTPUT_PSF = f'{OUTPUT_TS}_full_25us.psf'

In [9]:
# generate filepath
n100_dir = f'/gluster/data/next/files/TOPOLOGY_John/N100_data/{RUN_NUMBER}/prod_psf/eutropia/{TIMESTAMP}/merged_psfs/final/'

In [11]:
# load it in
print(f'Loading in PSF:\n{n100_dir}{PSF_name}')
psf = load_psf(f'{n100_dir}{PSF_name}')
display(psf)

Loading in PSF:
/gluster/data/next/files/TOPOLOGY_John/N100_data/14914/prod_psf/eutropia/010525/merged_psfs/final/010525_full_25us.psf


,xr,yr,zr,x,y,z,factor,nevt
0,-99.5,-99.5,0.0,0.0,0.0,12.5,0.000000,33
1,-99.5,-99.5,0.0,0.0,0.0,37.5,0.000064,90
2,-99.5,-99.5,0.0,0.0,0.0,62.5,0.000000,96
3,-99.5,-99.5,0.0,0.0,0.0,87.5,0.000000,95
4,-99.5,-99.5,0.0,0.0,0.0,112.5,0.000079,125
...,...,...,...,...,...,...,...,...
1919995,99.5,99.5,0.0,0.0,0.0,1087.5,0.000007,338
1919996,99.5,99.5,0.0,0.0,0.0,1112.5,0.000077,362
1919997,99.5,99.5,0.0,0.0,0.0,1137.5,0.000022,361
1919998,99.5,99.5,0.0,0.0,0.0,1162.5,0.000063,381


In [14]:
# downsample that shit
psf_ds = downsample_PSF(psf, downsample)
display(psf_ds)

,xr,yr,zr,x,y,z,factor,nevt
0,-100.0,-100.0,0.0,0.0,0.0,12.5,1.340718e-04,140
1,-100.0,-100.0,0.0,0.0,0.0,37.5,4.162700e-05,358
2,-100.0,-100.0,0.0,0.0,0.0,62.5,6.513766e-06,383
3,-100.0,-100.0,0.0,0.0,0.0,87.5,5.754310e-07,433
4,-100.0,-100.0,0.0,0.0,0.0,112.5,1.974398e-05,460
...,...,...,...,...,...,...,...,...
307195,97.5,97.5,0.0,0.0,0.0,1087.5,6.606906e-05,3169
307196,97.5,97.5,0.0,0.0,0.0,1112.5,7.843248e-05,3258
307197,97.5,97.5,0.0,0.0,0.0,1137.5,6.234791e-05,3210
307198,97.5,97.5,0.0,0.0,0.0,1162.5,7.359585e-05,3345


In [18]:
# generate file structure to store
output_dir = f'/gluster/data/next/files/TOPOLOGY_John/N100_data/{RUN_NUMBER}/prod_psf/eutropia/{OUTPUT_TS}/merged_psfs/final/'
print(f'Generating file structure at:\n{output_dir}')
output_Path = Path(output_dir)
output_Path.mkdir(parents=True, exist_ok=True)

Generating file structure at:
/gluster/data/next/files/TOPOLOGY_John/N100_data/14914/prod_psf/eutropia/190525/merged_psfs/final/


In [20]:
# store
write_psf(psf_ds, output_dir, OUTPUT_PSF)